In [1]:
# Rotate IP addresses
import requests as r
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import tweepy, json

import nltk
from nltk import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jetthollister/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Tweepy info

CONSUMER_KEY = 'pUHzK7aVAOIP8LtT71IkLYmUJ'
CONSUMER_SECRET = 'R59s8VEHOCVkcyOHBZShRpE9aJ6wCqNzGiKyEILJ7KFgqu8zXx'
ACCESS_KEY = '1498751586-6eQO2nqrFzs1541fhahjCUOZQWIwQQK3BfUmmuh'
ACCESS_SECRET = 'wFHGqYDNMs0iJVvoocHH2uJINyQOXiStzvoGmyUn5KE3O'
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
api = tweepy.API(auth)

In [7]:
trending = []
queue = []

def manageTrends():
    global trending, queue
    
    # Tweet volume threshold is currently set to 100,000
    tweet_volume = 100000

    trends = [[trend['name'], trend['tweet_volume']] for trend in api.trends_place(id=2352824)[0]['trends'] 
              if type(trend['tweet_volume']) == int and trend['tweet_volume'] >= tweet_volume
              and trend['promoted_content'] == None]
    
    for obj in trends:
        if obj[0] not in trending:
            queue.append(obj)
    
    return
    

In [8]:
manageTrends()

#trend = queue.pop(0) add this in for production
trend = queue[0]

In [9]:
queue

[['Twitch', 477351],
 ['Rand Paul', 111247],
 ['Liverpool', 141805],
 ['Bitcoin', 267367]]

In [10]:
trend

['Twitch', 477351]

In [11]:
# Get articles based on topics

def getInfo(trend, page):
    global titles
    global links
    global summaries
    
    today = datetime.today().strftime('%Y-%m-%d')
    key = '878293ce72msha47261c9511818ep1f0a27jsn0d54eeb13464'
    url = "https://rapidapi.p.rapidapi.com/v1/search"
    querystring = {"q":trend,
                   "sort_by":"relevancy","lang":"en",
                   'page_size': 5,
                   "page":page,
                   'country': 'US',
                   'from': today}

    headers = {
        'x-rapidapi-key': key,
        'x-rapidapi-host': "newscatcher.p.rapidapi.com"
        }
    re = r.request("GET", url, headers=headers, params=querystring)
    
    for item in re.json()['articles']:
        if item['link'] not in links:
            titles.add(item['title'])  
            links.add(item['link'])
            summaries.add(item['summary'])

    return titles

In [15]:
titles = set()
links = set()
summaries = set()

for i in range(1, 5):
    getInfo(trend, i)

In [16]:
from gensim.summarization import summarize

def getSummary(group):

    summary = sent_tokenize(summarize(' '.join(group), ratio=0.1))[0]

    return summary.strip()

getSummary(summaries)

'Well, thatboth doses of the COVID-19 vaccine so we can get back to doing what we love: dancing.Thankfully, Movement producers Paxahau are playing Santa by delivering two-days of heart-thumping sets from acclaimed local beatmakers and celebrated international performers thanks to a partnership with streaming platform Twitch.Launched in September — you know, when we were supposed to be raving IRL — Movement Selects Vol.'

In [17]:
summaries

{'',
 "A huge audience turned to Twitch to get more of their entertainment in November, according to data from Arsenal.gg — as pointed out by a StreamElements analysis. Viewers watched nearly 1.7 billion hours of Twitch content during the month. That is up slightly over October. Facebook Gaming's video viewership more than doubled year over year, though this comes with the caveat that Facebook has made dubious claims in the past about the size of its video audience. While gaming is still the core of Twitch's business, the platform is growing by expanding its appeal.",
 'Allied Esports, a global esports entertainment company ("Allied Esports" or the "Company") and a subsidiary of Allied Esports Entertainment, Inc. (AESE) - Get Report, has announced the launch of a 24-hour cross-channel content strategy on Twitch. Hundreds of hours of Allied Esports\' tournament productions and exclusive original content will be programmed around the clock across the Company\'s Twitch channels, including

In [93]:
proxies = getProxies()
agg = 0
news = ''

for i, link in enumerate(links[0:3]):
    temp = scrapeText(link, proxies)
    if len(temp) >= 100:
        news += ' ' + temp
    print(len(temp), link)

Error: missing text
0 https://www.newsweek.com/barr-tells-federal-prosecutors-investigate-clear-instances-voting-irregularities-sidestepping-1546180
Scrape completed.
10737 https://www.citizensvoice.com/news/wilkes-barre-officials-negotiating-lease-of-hollenback-golf-course/article_3501e415-d471-58e2-a6a7-6243690900a8.html
Scrape completed.
988 https://kvoa.com/ap-national-news/2020/11/09/barr-oks-election-probes-despite-little-evidence-of-fraud/


In [92]:
error_count = 0

def getProxies():
    global error_count
    url = 'https://www.us-proxy.org/'
    try:
        req = r.get(url, headers={}, timeout=10)
        src = req.content
        soup = BeautifulSoup(src, 'lxml')
        proxy = soup.find('td').getText()
        
        proxies = {'http': proxy}
        return proxies

    except (r.exceptions.Timeout, r.ConnectionError):
        error_count += 1
        if error_count < 5:
            print('Error - retrying...') 
            return getProxies()
        else:
            print('Error - tries exceeded') 
            return
            
def scrapeText(url, proxies):
    global agg
    
    try:
        req = r.get(url, proxies=proxies, timeout=10)
        src = req.content
        soup = BeautifulSoup(src, 'lxml')
        
        try:
            text = soup.find('article').getText()
            print('Scrape completed.')
            agg += 1
            return text
        except:
            print('Error: missing text')
            return ''

    except (r.exceptions.Timeout, r.ConnectionError):
        print(f'Error: {url}')
        return ''